# Data Wrangling: Merging Article Quality Scores With Population Counts

To generate the final csv file for this project, we must merge the intermediate article quality score by revision id file, the politican by revision id file, the population by country and region file, and the politician by country file. This notebook goes through the steps required to merge and format these csv files.

## Step 1: Read in Files

The following packages are required to import for this notebook:

In [2]:
import pandas as pd

In [3]:
population_by_country_region = pd.read_csv("../../data/intermediate/population_by_country_region_AUG.2024.csv")
politician_by_country = pd.read_csv("../../data/intermediate/politicians_by_country_AUG.2024.csv")
revision_id_by_article_quality = pd.read_csv("../../data/intermediate/politicians_by_revid_article_score_OCT.2024.csv")
politician_by_revision_id = pd.read_csv("../../data/intermediate/politicians_by_revid_OCT.2024.csv")

In [4]:
revision_id_by_article_quality = revision_id_by_article_quality.drop('Unnamed: 0', axis = 1)
population_by_country_region = population_by_country_region.drop('Unnamed: 0', axis = 1)

## Step 2: Finding the Revision ID by Article Title

Here, we seek to unify the revision_id_by_article_quality file with the politician_by_revision_id file to match article titles with revision ids and article quality. Since there are non unique revision IDs (one politician corresponding to multiple countries), we deduplicate these before joining.

In [5]:
#filter to revids found in article quality and drop duplicates
politician_by_revision_id = politician_by_revision_id[politician_by_revision_id.revision_id.isin(revision_id_by_article_quality.revision_id)].drop_duplicates()
#drop duplicates
revision_id_by_article_quality = revision_id_by_article_quality.drop_duplicates()

In [6]:
#merge the two dataframes on revision id
article_title_by_revision_id = pd.merge(left = politician_by_revision_id, 
                                        right = revision_id_by_article_quality, 
                                        how = "left",
                                        on = "revision_id")

## Step 3: Finding the Country for A Given Politician and Revision ID

Next, we have to join the article_title_by_revision_id file with the politicians by country, so we can get the countries corresponding to an article title, revision id, and article quality.

In [7]:
#url not required in final csv
politician_by_country = politician_by_country.drop("url", axis = 1)
#rename columns for merge
politician_by_country.columns = ["article_title", "country"]

In [8]:
#merge the two dataframes on article title
#account for article title having multiple countries by left join with politician first
article_title_by_revision_id_country = pd.merge(left = politician_by_country, 
                                                right = article_title_by_revision_id, 
                                                how = "left",
                                                on = "article_title")

## Step 4: Merging on Country to Find Regions

Finally, we merge on country to get population and region estimates for each article. First we rename the population by country region table to match the join id needed and the final column name in the output csv.

In [9]:
#rename columns
population_by_country_region.columns = ["country", "population", "region"]

Next we identify the superset of countries, so we can compare the final list of mapped countries and identify which are missing after the join.

In [10]:
#find unique country list for each table
article_title_countries = list(set(article_title_by_revision_id_country.country.tolist()))
population_countries = list(set(population_by_country_region.country.tolist()))
#find superset of unique countries across tables
all_country_set = list(set(article_title_countries + population_countries))

We merge the two tables on country, and only preserve the records shared across the two tables with an inner join. We save this output csv as requested by the document.

In [11]:
#merge the two dataframes on article title
#inner join to preserve only records which match across both
wp_politicians_by_country = pd.merge(left = article_title_by_revision_id_country, 
                                    right = population_by_country_region, 
                                    how = "inner",
                                    on = "country")
wp_politicians_by_country.to_csv("../../data/final/wp_politicians_by_country.csv", index = False)

Now we write out the countries which have no match.

In [12]:
#List of final countries
final_country_list = list(set(wp_politicians_by_country.country.tolist()))
#List of countries with no match
no_match_list = []
for country in all_country_set:
    if country not in final_country_list:
        no_match_list.append(country)
#write out file 
with open("../../data/final/wp_countries-no_match.txt", "w") as file:
    for country in no_match_list:
        file.write(f"{country}\n")